### Imports

In [26]:
import mltfm.generate_data
from DL_TFM.scripts.tracNet import TracNet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import keras

Set seeds for reproducability.

In [24]:
random_seed = 2
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [29]:
device = torch.device('cpu')

### Evaluation

In this notebook I will compare the performance of five approaches to TFM, namely

    1. TracNet_MATLAB by Wang & Lin
    2. TracNet_Pytorch_SGD by Richard & Sabaß
    3. TracNet_Pytorch_ADAM by Richard & Sabaß
    4. Easy-to-use_TFM_package by Sabaß
    5. Noisy_UNet by TU Dortmund
    
The test data includes both artificially created, labeled data from several research groups and experimental data. First, load the Pytorch and Keras based models.

In [31]:
TracNet_Pytorch_SGD = TracNet(n_channels=1).double().to(device)
TracNet_Pytorch_SGD.load_state_dict(torch.load('DL_TFM/models/TracNet104_best_val_rmse_2.601_SGD.pth'), strict=False)

<All keys matched successfully>

In [32]:
TracNet_Pytorch_ADAM = TracNet(n_channels=1).double().to(device)
TracNet_Pytorch_ADAM.load_state_dict(torch.load('DL_TFM/models/TracNet104_best_val_rmse_1.537_ADAM.pth'), strict=False)

<All keys matched successfully>

In [33]:
Noisy_UNet = keras.models.load_model('mltfm/models/model_noise_1e-3.h5')

## 1. Artificial test data from TU Dortmund

In [34]:
import h5py

fn = h5py.File('displacement.h5')
list(fn.keys())

get_ipython().run_line_magic('matplotlib', 'notebook')

fig, ax = plt.subplots()
fig.tight_layout(pad=3, w_pad=3, h_pad=3)
ax.quiver(fn['data'][0,:,:,0], fn['data'][0,:,:,1], scale=1.2)
ax.set_title('Displacement field: Strain vectors', {'fontsize': 11})

OSError: Unable to open file (unable to lock file, errno = 35, error message = 'Resource temporarily unavailable')